### Setup 
##### No API Key needed - MTGJSON is open and free to use!

**Mac**
```bash
python -m venv .venv
source .venv/bin/activate 
pip install -r requirements.txt
jupyter notebook
```
**Windows**
```bash
python -m venv .venv
source .venv\Scripts\activate
pip install -r requirements.txt
jupyter notebook
```

In [1]:
import requests
import zipfile
import io
import os
import datetime
import json   
import pandas as pd
import numpy as np
import re

In [ ]:
#Setting up some global variables and provenance recording
PROVENANCE_FILE = "provenance.json"
OUTPUT_DIR = "data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def record_provenance(step_name, description, inputs=None, outputs=None, extra=None):
    entry = {
        "timestamp": datetime.datetime.now().isoformat() + "Z",
        "step": step_name,
        "description": description,
        "inputs": inputs or [],
        "outputs": outputs or [],
        "extra": str(extra) if extra is not None else None
    }

    if not os.path.exists(PROVENANCE_FILE):
        with open(PROVENANCE_FILE, "w") as f:
            json.dump({"workflow": "MTGJSON curation", "records": []}, f, indent=2)

    with open(PROVENANCE_FILE, "r+") as f:
        data = json.load(f)
        data["records"].append(entry)
        f.seek(0)
        json.dump(data, f, indent=2)
        f.truncate()

    print(f"Recorded provenance: {step_name}")


In [ ]:
#Download AllPrintings
ALL_PRINTINGS = "https://mtgjson.com/api/v5/AllPrintings.json.zip"
response = requests.get(ALL_PRINTINGS)
response.raise_for_status() 

zip_path = os.path.join(OUTPUT_DIR, "AllPrintings.json.zip")
with open(zip_path, "wb") as f:
    f.write(response.content)

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall(OUTPUT_DIR)

record_provenance(
    "download_allprintings", 
    "Downloaded AllPrintings.json from MTGJSON", 
    ["https://mtgjson.com/api/v5/AllPrintings.json.zip"],
    [os.path.join(OUTPUT_DIR, "AllPrintings.json")]
)

Recorded provenance: download_allprintings


In [ ]:
#Download AllSetFiles
ALL_SETS = "https://mtgjson.com/api/v5/AllSetFiles.zip"

response = requests.get(ALL_SETS)
response.raise_for_status()

zip_path = os.path.join(OUTPUT_DIR, "AllSetFiles.zip")
with open(zip_path, "wb") as f:
    f.write(response.content)

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(OUTPUT_DIR)

record_provenance(
    "download_allsets", 
    "Downloaded AllSetFiles from MTGJSON",
    ["https://mtgjson.com/api/v5/AllSetFiles.zip"],
    [os.path.join(OUTPUT_DIR, "*.json")],
    "Includes all .json files except AllPrintings.json"
)

Recorded provenance: download_allsets


In [ ]:
#Create dictionary of all sets with filtered attributes
skip_files = {"AllSetFiles.zip", "AllPrintings.json", "AllPrintings.json.zip"}
sets = {}

for filename in os.listdir(OUTPUT_DIR):
    if not filename.endswith(".json") or filename in skip_files:
        continue

    path = os.path.join(OUTPUT_DIR, filename)
    with open(path, "r", encoding="utf-8") as f:
        json_data = json.load(f)
        set_data = json_data["data"]

        sets[set_data["code"]] = {
            "name": set_data["name"],
            "releaseDate": set_data.get("releaseDate"),
            "type": set_data.get("type"),
        }


record_provenance(
    "all_sets_dict", 
    "Created a dictionary of all sets include only relevant set attributes",
    [os.path.join(OUTPUT_DIR, "*.json")],
    ["sets dictionary"],
    "Excluded AllSetFiles.zip, AllPrintings.json, and AllPrintings.json.zip from input"
)

Recorded provenance: all_sets_dict


In [ ]:
#Creating set code list of Universe Beyond cards
tables = pd.read_html('https://mtg.fandom.com/wiki/Universes_Beyond')
ub_table = tables[0]
ip_list = ub_table["IP"].dropna().tolist()

clean_ip_list = []
for ip in ip_list:
    cleaned = re.sub(r'(?:\s*\[\d+\])+$', '', ip)
    parts = [p.strip() for p in cleaned.split('/') if p.strip()]
    clean_ip_list.extend(parts)

ip_to_set = {}

for ip in clean_ip_list:
    match = next(
        (code for code, info in sets.items()
         if ip.lower() in info.get("name", "").lower()),
        None
    )
    if match:
        ip_to_set[ip] = match

ip_set_codes = ip_to_set.values()

print("Matched:", len(ip_to_set), "of", len(clean_ip_list))
print(ip_set_codes)

record_provenance(
    "set_code_list",
    "Created a list of set codes for Universe Beyond cards from MTG's Fandom.com wiki",
    ["https://mtg.fandom.com/wiki/Universes_Beyond", "sets dictionary"],
    ["ip_to_set dictionary", "ip_set_codes list"],
    "Removed citation markers like [1], split multi-franchise names, and matched against MTG set names"
)

Matched: 10 of 28
dict_values(['40K', 'BOT', 'MLTR', 'WHO', 'REX', 'FCLU', 'PIP', 'MACR', 'LMAR', 'AFIC'])
Recorded provenance: set_code_list


In [7]:
print(next(iter(sets)))
print(json.dumps(sets[next(iter(sets))], indent=4))

40K
{
    "name": "Warhammer 40,000 Commander",
    "releaseDate": "2022-10-07",
    "type": "commander"
}


In [ ]:
#Filtering universe beyond (UB) cards and non-UB cards into separate dataframes
with open("data/AllPrintings.json", "r", encoding="utf-8") as f:
    ap_data = json.load(f)["data"]

cards = []
universes_beyond_cards = []

for set_code, set_data in ap_data.items():
    for card in set_data["cards"]:
        meta = sets.get(set_code, {})
        card["releaseDate"] = meta.get("releaseDate")
        card["setType"] = meta.get("type")
        card["setName"] = meta.get("name")
        card["setCode"] = set_code
        card["isUniversesBeyond"] = set_code in ip_set_codes

        cards.append(card)

df = pd.DataFrame(cards)

record_provenance(
    "merge_set_metadata",
    "Merged set-level metadata into card data from AllPrintings.json to create a unified dataset",
    ["data/AllPrintings.json", "sets dictionary", "ip_set_codes list"],
    ["df (merged dataframe)"],
    "Attached set name, type, releaseDate, and isUniversesBeyond flag fields to each card record"
)

Recorded provenance: merge_set_metadata


In [ ]:
#Data cleaning - coercing column types
for col in ["manaValue", "power", "toughness", "defense", "edhrecRank"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df["releaseDate"] = pd.to_datetime(df["releaseDate"], errors="coerce")

for col in ["name", "type", "rarity", "setName", "setCode", "text"]:
    df[col] = df[col].astype(str)

for col in ["keywords", "colors"]:
    df[col] = df[col].apply(
        lambda x: [str(i) for i in x] if isinstance(x, list)
        else ([] if pd.isna(x) else [str(x)])
    )

record_provenance(
    "coercing_data_types",
    "Ensured all columns are coerced to their respective types for easier future use",
    ["df (merged dataframe)"],
    ["df (cleaned, merged dataframe)"]
)

Recorded provenance: coercing_data_types


In [ ]:
#Adding helpful columns
df["keywordCount"] = df["keywords"].apply(lambda x: len(x) if isinstance(x, list) else 0)
df["baseLevel"] = df[["manaValue", "power", "toughness"]].fillna(0).sum(axis=1)
df["isMulticolor"] = df["colors"].apply(lambda x: len(x) > 1 if isinstance(x, list) else False)

record_provenance(
    "adding_cols",
    "Adding keywordCount, baseLevel, and isMulticolor",
    ["df (cleaned, merged dataframe)"],
    ["df (supplementary columns)"],
)

Recorded provenance: adding_cols


In [ ]:
#Splitting data into Universe Beyond and non-Universe Beyond dataframes

df_ub = df[df["isUniversesBeyond"] == True]
df_non_ub = df[df["isUniversesBeyond"] == False]

record_provenance(
    "separate_dataframes", 
    "Merged set metadata into every card and separated Universes Beyond vs non-UB cards using known ip_set_codes",
    ["df (supplementary columns)"],
    ["df_ub (Universe Beyond dataframe)", "df_non_ub (non-Universe Beyond dataframe)"]
)

Recorded provenance: separate_dataframes


In [ ]:
#Filtering both dataframes to keep relevant columns only
cols_to_keep = ["name", "type", "rarity", "setName", "setCode", "isUniversesBeyond", "releaseDate", "keywords", "keywordCount", "manaValue", "power", "toughness", "baseLevel", "defense", "colors", "isMulticolor", "text", "edhrecRank"]
df_ub = df_ub.reindex(columns=cols_to_keep, fill_value=None)
df_non_ub = df_non_ub.reindex(columns=cols_to_keep, fill_value=None)

record_provenance(
    "filter_dataframes", 
    "Retain only potentially relevant columns",
    ["df_ub (Universe Beyond dataframe)", "df_non_ub (non-Universe Beyond dataframe)"],
    ["df_ub (filtered Universe Beyond dataframe)", "df_non_ub (filtered non-Universe Beyond dataframe)"]
)

Recorded provenance: filter_dataframes


In [13]:
print(df_non_ub.head(3))
print("############################################")
df_non_ub.iloc[0]
# print(df_ub.head(3))

                name                     type    rarity        setName  \
0  Ancestor's Chosen  Creature — Human Cleric  uncommon  Tenth Edition   
1  Ancestor's Chosen  Creature — Human Cleric  uncommon  Tenth Edition   
2     Angel of Mercy         Creature — Angel  uncommon  Tenth Edition   

  setCode  isUniversesBeyond releaseDate        keywords  keywordCount  \
0     10E              False  2007-07-13  [First strike]             1   
1     10E              False  2007-07-13  [First strike]             1   
2     10E              False  2007-07-13        [Flying]             1   

   manaValue  power  toughness  baseLevel  defense colors  isMulticolor  \
0        7.0    4.0        4.0       15.0      NaN    [W]         False   
1        7.0    4.0        4.0       15.0      NaN    [W]         False   
2        5.0    3.0        3.0       11.0      NaN    [W]         False   

                                                text  edhrecRank  
0  First strike (This creature deals c

name                                                 Ancestor's Chosen
type                                           Creature — Human Cleric
rarity                                                        uncommon
setName                                                  Tenth Edition
setCode                                                            10E
isUniversesBeyond                                                False
releaseDate                                        2007-07-13 00:00:00
keywords                                                [First strike]
keywordCount                                                         1
manaValue                                                          7.0
power                                                              4.0
toughness                                                          4.0
baseLevel                                                         15.0
defense                                                            NaN
colors

In [ ]:
#Data cleaning
for df in [df_ub, df_non_ub]:
    df["name"] = df["name"].str.strip().str.lower()
    df["rarity"] = df["rarity"].str.lower()
    df["type"] = df["type"].str.lower()
    df["text"] = (
        df["text"]
        .str.replace("—", "-", regex=False)
        .str.replace("–", "-", regex=False)
        .str.replace("’", "'", regex=False)
        .str.replace("“", '"', regex=False)
        .str.replace("”", '"', regex=False)
    )

record_provenance(
    "data_cleaning", 
    "Normalized important string fields", 
    ["df_ub (filtered Universe Beyond dataframe)", "df_non_ub (filtered non-Universe Beyond dataframe)"], 
    ["df_ub (cleaned Universe Beyond dataframe)", "df_non_ub (cleaned non-Universe Beyond dataframe)"]
)

Recorded provenance: data_cleaning


In [15]:
df_ub.head()

,name,type,rarity,setName,setCode,isUniversesBeyond,releaseDate,keywords,keywordCount,manaValue,power,toughness,baseLevel,defense,colors,isMulticolor,text,edhrecRank
2675,"szarekh, the silent king",legendary artifact creature — necron,mythic,"Warhammer 40,000 Commander",40K,True,2022-10-07,"[Flying, Mill, My Will Be Done]",3,4.0,3.0,4.0,11.0,NaN,[B],False,"Flying\nMy Will Be Done - Whenever Szarekh, th...",9092.0
2676,abaddon the despoiler,legendary creature — astartes warrior,mythic,"Warhammer 40,000 Commander",40K,True,2022-10-07,"[Mark of Chaos Ascendant, Trample]",2,5.0,5.0,5.0,15.0,NaN,"[B, R, U]",True,Trample\nMark of Chaos Ascendant - During your...,8018.0
2677,inquisitor greyfax,legendary creature — human inquisitor,mythic,"Warhammer 40,000 Commander",40K,True,2022-10-07,"[Hunt for Heresy, Investigate, Unquestionable ...",4,4.0,3.0,3.0,10.0,NaN,"[B, U, W]",True,Vigilance\nUnquestionable Wisdom - Other creat...,8664.0
2678,the swarmlord,legendary creature — tyranid,mythic,"Warhammer 40,000 Commander",40K,True,2022-10-07,"[Rapid Regeneration, Xenos Cunning]",2,6.0,5.0,5.0,16.0,NaN,"[G, R, U]",True,Rapid Regeneration - The Swarmlord enters with...,11794.0
2679,imotekh the stormlord,legendary artifact creature — necron,mythic,"Warhammer 40,000 Commander",40K,True,2022-10-07,"[Grand Strategist, Phaeron]",2,4.0,3.0,3.0,10.0,NaN,[B],False,Phaeron - Whenever one or more artifact cards ...,4364.0


In [ ]:
#Exporting to CSV
df_ub.to_csv("output/universes_beyond.csv", index=False)
df_non_ub.to_csv("output/non_universes_beyond.csv", index=False)

record_provenance(
    "export_csv", 
    "Exported both dataframes to CSV for easier future use",
    ["df_ub (cleaned Universe Beyond dataframe)", "df_non_ub (cleaned non-Universe Beyond dataframe)"],
    ["output/universes_beyond.csv", "output/non_universes_beyond.csv"]
)

Recorded provenance: export_csv
